In [ ]:
!pip3 install seaborn
!pip3 install sklearn
!pip3 install keras
!pip3 install pandas

In [1]:
import pandas as pd
import numpy as np
import numpy as np
import seaborn as sns; sns.set(color_codes=True)
import re 
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load data from csv
df = pd.read_csv('StandUpData.csv')
df.loc[df.language == 'en']

df.head()

,title,tags,date_posted,link,name,year,transcript,language,runtime,rating
0,Chris D’Elia: Man on Fire,"Chris D'Elia,Stand-up transcripts","April 19th, 2020",https://scrapsfromtheloft.com/2020/04/19/chris...,Chris D'Elia,2017.0,Fire Fire Man on fire Fire Fire Fire Man on ...,en,65.0,6.6
1,George Carlin: The Indian Drill Sergeant,"George Carlin,Stand-up transcripts","April 13th, 2020",https://scrapsfromtheloft.com/2020/04/13/georg...,George Carlin,NaN,"In The Indian Sergeant, was emerging as George...",en,NaN,NaN
2,Tom Segura: Ball Hog,"Stand-up transcripts,Tom Segura","March 25th, 2020",https://scrapsfromtheloft.com/2020/03/25/tom-s...,Tom Segura,2020.0,"How you doing? No shit. Thank you, thank you...",en,70.0,7.3
3,Bert Kreischer: Hey Big Boy,"Bert Kreischer,Stand-up transcripts","March 21st, 2020",https://scrapsfromtheloft.com/2020/03/21/bert-...,Bert Kreischer,2020.0,Ladies and gentlemen Bert Kreischer! Yeah! Wh...,en,62.0,7.1
4,Bert Kreischer: Fighting a Bear,"Bert Kreischer,Stand-up transcripts","March 21st, 2020",https://scrapsfromtheloft.com/2020/03/21/bert-...,Bert Kreischer,NaN,"The machine, Mr. Bert Kreischer, everybody. Le...",en,NaN,NaN


In [3]:
# Remove leading, trailing, and intermitent whitespace
print("With extra whitespace: ")
print(df.iloc[0].transcript[:100])
df.transcript = df.transcript.apply(lambda x: re.sub("\s+", " ", x).strip())
print("Without extra whitespace:")
print(df.iloc[0].transcript[:100])

With extra whitespace: 
 Fire Fire  Man on fire Fire Fire Fire Man on fire Yes. Yes. All right. Yeah. All right. Just relax.
Without extra whitespace:
Fire Fire Man on fire Fire Fire Fire Man on fire Yes. Yes. All right. Yeah. All right. Just relax. W


In [4]:
# Collect all charcters to see which ones are present in the transcripts
chars = []
df.transcript.apply(lambda x: [chars.append(char) for char in x if char not in chars])

print(len(chars))
print(chars)

123
['F', 'i', 'r', 'e', ' ', 'M', 'a', 'n', 'o', 'f', 'Y', 's', '.', 'A', 'l', 'g', 'h', 't', 'J', 'u', 'x', 'W', 'c', 'k', ',', 'd', 'C', 'I', 'm', 'p', 'y', 'b', 'L', 'w', 'U', '?', 'z', 'H', 'E', 'T', ':', 'j', 'v', 'N', 'R', 'S', 'O', 'B', '-', 'P', 'q', 'D', '!', 'V', 'G', 'K', '–', '—', ';', '/', 'Z', "'", 'é', 'ú', '%', '*', 'Q', '$', 'X', '#', '″', '£', '&', 'ç', 'í', 'ö', 'ä', 'ô', 'ñ', 'Ç', 'è', '@', 'ü', 'Ü', 'ó', '¿', 'á', 'É', 'Ó', '×', '\\', ']', '¡', '_', 'å', 'à', '®', 'ê', 'ù', 'Ã', 'ƒ', 'Â', 'º', '`', '+', 'Á', 'ì', 'È', 'ò', '′', '¶', 'û', '♫', 'Ö', '¢', 'ﬂ', '♬', '»', '=', '\u200b', 'â', '~', '>']


In [5]:
# Remove all unwanted characters
remove = ['-', '–', '—', '/', 'é', 'ú', '%', '*', '$', '#', '″', '£', '&', 'ç', 'í', 'ö', 'ä', 'ô', 'ñ', 'Ç', 'è', '@', 'ü', 'Ü', 'ó', '¿', 'á', 'É', 'Ó', '×', '\\', ']', '¡', '_', 'å', 'à', '®', 'ê', 'ù', 'Ã', 'ƒ', 'Â', 'º', '`', '+', 'Á', 'ì', 'È', 'ò', '′', '¶', 'û', '♫', 'Ö', '¢', 'ﬂ', '♬', '»', '=', '\u200b', 'â', '~', '>']
df.transcript = df.transcript.apply(lambda x: ''.join([char for char in x if char not in remove]))

In [6]:
text = ''
for i in df.transcript:
    text += (i + ' ')

len(text)

13166511

In [7]:
# Shorten text by 1/10 to save processing time
portion = int(len(text)/10)
text = text[:portion]
len(text)

1316651

In [8]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

# Turn 40-character chunks into vectors for the x. y is the next character in the original sequence
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# Build the model: 3 layers deep LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(len(chars), activation='softmax'))

optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # Helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(150):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print('\n')

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
# Checkpoint save on new best
filepath = "model_best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

model.summary()

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback, checkpoint])

Using TensorFlow backend.


total chars: 60
nb sequences: 438871
Vectorization...
Build model...
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 40, 256)           324608    
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 40, 256)           525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 40, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
___________________________________________________

KeyboardInterrupt: 

In [10]:
# Load the model from h5 file
from keras.models import load_model

filename = "model_best.h5"
model = load_model(filename)
optimizer = Adam(lr=0.0001) 
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback, checkpoint])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 40, 256)           324608    
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 40, 256)           525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 40, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 60)               

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: RuntimeWarning: divide by zero encountered in log


 And I would

----- diversity: 1.2
----- Generating with seed: "meaty. Always a Always a little meanlook"
meaty. Always a Always a little meanlook and final night, weve all works about I said is when Im in the mirrarity shates, right? There we do. And SNA, Im gonna nine. Its not like hes wearing


Epoch 00002: loss improved from 1.09473 to 1.07340, saving model to model_best.h5
Epoch 3/60
438871/438871 [==============================] - 567s 1ms/step - loss: 1.0625

----- Generating text after Epoch: 2
----- diversity: 0.2
----- Generating with seed: "es or dreams or supportive friends, fami"
es or dreams or supportive friends, family, I dont know what they did that shit. I was like, This is a little bit different thing. She said, I dont know what I mean? I dont know if you want 

----- diversity: 0.5
----- Generating with seed: "es or dreams or supportive friends, fami"
es or dreams or supportive friends, family, the shows the only thing that was a car company. I was like, You know, p

KeyboardInterrupt: 

In [17]:
# Load the model from h5 file
from keras.models import load_model

filename = "model_best.h5"
model = load_model(filename)
optimizer = Adam(lr=0.00001) 
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback, checkpoint])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 40, 256)           324608    
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 40, 256)           525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 40, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 60)               

KeyboardInterrupt: 

In [12]:
# # Load the model from h5 file
# from keras.models import load_model

# filename = "model_best.h5"
# model = load_model(filename)
# optimizer = Adam(lr=0.0001) 
# model.compile(loss='categorical_crossentropy', optimizer=optimizer)
# model.summary()

# model.fit(x, y,
#           batch_size=128,
#           epochs=60,
#           callbacks=[print_callback, checkpoint])

In [ ]:
# # Translate model input/output with dictionaries
# chars = sorted(list(set(text)))
# print('total chars:', len(chars))
# char_indices = dict((c, i) for i, c in enumerate(chars))
# indices_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
# start_index = random.randint(0, len(text) - maxlen - 1)
# for diversity in [0.2, 0.5, 1.0, 1.2]:
#     print('----- diversity:', diversity)

#     generated = ''
#     sentence = text[start_index: start_index + maxlen]
#     generated += sentence
#     print('----- Generating with seed: "' + sentence + '"')
#     sys.stdout.write(generated)

#     for i in range(200):
#         x_pred = np.zeros((1, maxlen, len(chars)))
#         for t, char in enumerate(sentence):
#             x_pred[0, t, char_indices[char]] = 1.

#         preds = model.predict(x_pred, verbose=0)[0]
#         next_index = sample(preds, diversity)
#         next_char = indices_char[next_index]

#         sentence = sentence[1:] + next_char

#         sys.stdout.write(next_char)
#         sys.stdout.flush()
#     print('\n')